The task is to price a bond that follows a stochastic interest rate and pays coupons.
$$
dr = \kappa (\theta e^{\mu t} -r)dt + \sigma r^\beta dW_t = f(r,t)dt + v(r,t)dW_t
$$
We use the notation of times $\{t_k\}_{k=0}^{k=K}$ and $\Delta t= \frac{T}{K}$. The euler maruyama scheme gives us:
$$
r(t_k)-r(t_{k-1})  = f(r(t_{k-1},t_{k-1}))\Delta t +v(r(t_{k-1},t_{k-1}))\sqrt{\Delta t}\phi_{k-1}
$$

We know that our bond pays $F$, the face value, at maturity. We also know it pays dividends, at rate $Ce^{-\alpha t}$. Under the risk neutral measure, these need to be discounted, both at the rate:

$$
\exp\left(-\int_0^t r(s)ds\right)
$$

Applying this discounting, and noting that the value of $B(r_0,t=0)% is an expectation, we have the expectation that we want to compute, being:

$$
B(r_0,0) = \mathbb{E}\left[
    \int_0^TCe^{-\alpha t}exp\left(\int_0^t r(s)ds\right)dt + Fexp\left(\int_0^T r(s)ds\right)
\right]
$$

Where each coupon needs to be discounted to zero at each timestep it is paid.

This is discretised fairly simply:

$$
\tilde{g} = \sum_{i=0}^{K-1} Ce^{-\alpha t_i}\Delta t \exp\left(-\sum_{j=0}^i r_j\Delta t\right) + F \exp\left(-\sum_{j=0}^Kr_j\Delta t\right)
$$

And the bond, through convergence in expectation, for N paths:

$$
B(r_0,t=0) \approx \frac{1}{N}\sum_{i=0}^{N-1} \tilde{g}
$$

In [1]:
import numpy as np
from numpy import exp
from scipy.stats import norm
from scipy.stats.qmc import Halton

In [2]:
T = 3
F = 240
theta = 0.0289
r0 = 0.0238
kappa = 0.09389
mu = 0.0141
C = 10.2
alpha = 0.01
beta = 0.418
sigma = 0.116


# 100,000 paths and 100 timesteps 

N=100000
K=100

delta_t = T/K

In [3]:
f = lambda r,t: kappa * (theta * np.exp(mu * t)-r)
v = lambda r, t: sigma * np.abs(r)** beta # modelling assumptions, interest rate is positive a.s.

def halton_sequence(N,K,scramble=True,seed=8032003):
    sampler = Halton(d=K,scramble=scramble,seed=seed)
    samples = sampler.random(N)
    return norm.ppf(samples)

def simulate_rate_paths(r0,T,K,N):
    delta_t =T/K
    t = np.linspace(0,T,K+1)
    rates = np.zeros((N,K+1))
    rates[:,0] = r0 #initial rate
    phi = halton_sequence(N, K)
    for k in range(1,K+1):
        rates[:,k] = rates[:,k-1] + f(rates[:,k-1],t[k-1]) * delta_t + v(rates[:,k-1],t[k-1]) * np.sqrt(delta_t) * phi[:,k-1]
    return rates, t, delta_t

In [4]:
def g(rates,t,delta_t):
    coupon_sum = 0.
    for i in range(len(t)-1):
        discount = exp(-np.sum(rates[:i+1] * delta_t))
        coupons = C * np.exp(-alpha * t[i]) * delta_t
        coupon_sum += coupons * discount
    final_discount = exp(-np.sum(rates*delta_t))
    return coupon_sum + F * final_discount

def B(rates,N,t,delta_t):
    payoffs = [g(rates[i], t, delta_t) for i in range(N)]
    return np.mean(payoffs),np.var(payoffs,ddof=1)



In [5]:
rates, t_grid, delta_t = simulate_rate_paths(r0, T, K, N)

# Compute the bond price using the function B
bond_price,var_bond = B(rates, N, t_grid, delta_t)

In [7]:
print(f'{bond_price:.4f}+/-{np.sqrt(var_bond)}')

252.3881+/-14.71928577306211
